In [1]:
import duckdb
import pandas as pd
from splink.duckdb.linker import DuckDBLinker

In [24]:
ferc_df.fuel_type_code_pudl.value_counts()

fuel_type_code_pudl
gas        11258
coal       10252
oil         3913
hydro       2764
solar       1058
nuclear      984
wind         721
waste         88
Name: count, dtype: Int64

In [46]:
preds = pd.read_parquet("supervised_predictions.parquet")

In [47]:
preds.head(1)

,match_weight,match_probability,source_dataset_l,source_dataset_r,record_id_l,record_id_r,plant_name_l,plant_name_r,gamma_plant_name,bf_plant_name,...,net_generation_mwh_r,gamma_net_generation_mwh,bf_net_generation_mwh,plant_id_pudl_l,plant_id_pudl_r,utility_id_pudl_l,utility_id_pudl_r,report_year_l,report_year_r,match_key
0,-1.584644,0.250041,eia_df,ferc_df,61768_2020_plant_total_6452,f1_steam_2020_12_56_8_1,interstate solar energy center,babcock preserve,0,0.089643,...,152720.0,3,126.736853,11958,9494.0,121.0,121.0,2020,2020,1


In [26]:
full_match_records = pd.read_parquet("full_match_records.parquet")

In [32]:
ferc_df.fuel_type_code_pudl.isnull().value_counts()

fuel_type_code_pudl
False    31038
True     25371
Name: count, dtype: int64

In [39]:
ferc_df.head(1)

,record_id,plant_name,utility_name,fuel_type_code_pudl,installation_year,construction_year,capacity_mw,net_generation_mwh,report_year,plant_id_pudl,utility_id_pudl
0,f1_steam_1994_12_14_1_4,mystic 7,boston edison company,oil,1975-01-01,1973-01-01,617.0,1946578.0,1994,1814,45


In [37]:
full_match_records.head(1)

,record_id_ferc1,record_id_eia,plant_id_eia,report_date,plant_part,generator_id,unit_id_pudl,prime_mover_code,energy_source_code_1,technology_description,...,opex_pumping,total_fuel_cost_ferc1,total_mmbtu_ferc1,fuel_type_code_pudl_ferc1,plant_id_report_year_ferc1,plant_id_report_year_util_id_ferc1,unit_heat_rate_mmbtu_per_mwh_ferc1,report_year,plant_id_pudl,utility_id_pudl
0,f1_gnrt_plant_2001_12_102_0_15,2196_2001_plant_total_12825,2196,2001-01-01,plant,<NA>,<NA>,IC,DFO,Petroleum Liquids,...,NaN,NaN,NaN,<NA>,<NA>_2001,<NA>_2001_<NA>,NaN,2001,1977,8378


In [45]:
gas_plants = ferc_df[ferc_df.fuel_type_code_pudl == "gas"]
gas_plants.merge(sorted_preds_df, left_on="record_id", right_on="record_id_r", how="left", indicator=True)._merge.value_counts()

_merge
both          15140
left_only      3944
right_only        0
Name: count, dtype: int64

In [53]:
merged_df = ferc_df.merge(sorted_preds_df, left_on="record_id", right_on="record_id_r", how="left", indicator=True)

In [54]:
merged_df.head(1)

,record_id,plant_name,utility_name,fuel_type_code_pudl,installation_year,construction_year,capacity_mw,net_generation_mwh,report_year,plant_id_pudl,...,gamma_net_generation_mwh,bf_net_generation_mwh,plant_id_pudl_l,plant_id_pudl_r,utility_id_pudl_l,utility_id_pudl_r,report_year_l,report_year_r,match_key,_merge
0,f1_steam_1994_12_14_1_4,mystic 7,boston edison company,oil,1975-01-01,1973-01-01,617.0,1946578.0,1994,1814,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [70]:
cov_dict = {}
for fuel in ["gas", "coal", "oil", "hydro", "solar", "nuclear", "wind", "waste"]:
    counts = merged_df[merged_df.fuel_type_code_pudl == fuel]._merge.value_counts()
    cov = counts["both"] / (counts["left_only"] + counts["both"])
    total_cap = merged_df[merged_df.fuel_type_code_pudl == fuel]["capacity_mw"].sum()
    cov_cap = merged_df[
        (merged_df.fuel_type_code_pudl == fuel) & (merged_df._merge == "both")
    ]["capacity_mw"].sum()
    perc_cap = cov_cap / total_cap
    cov_dict[fuel] = [round(cov, 2), round(perc_cap, 2)]


cov_df = pd.DataFrame(cov_dict).T
cov_df.columns=["% Record Coverage", "% Capacity Coverage"]

In [81]:
merged_df[(merged_df.utility_name.notnull()) & (merged_df._merge == "both") & (merged_df.match_probability != 1)].drop_duplicates(subset="record_id")

,record_id,plant_name,utility_name,fuel_type_code_pudl,installation_year,construction_year,capacity_mw,net_generation_mwh,report_year,plant_id_pudl,...,gamma_net_generation_mwh,bf_net_generation_mwh,plant_id_pudl_l,plant_id_pudl_r,utility_id_pudl_l,utility_id_pudl_r,report_year_l,report_year_r,match_key,_merge
76,f1_steam_2001_12_16_0_1,seabrook 1,canal electric company,nuclear,1990-01-01,1990-01-01,1197.000000,3.062438e+05,2001,2499,...,3.0,131.090317,2499.0,2499.0,9741.0,51.0,2001.0,2001.0,0,both
88,f1_steam_2002_12_16_0_1,seabrook 1,canal electric company,nuclear,1990-01-01,1990-01-01,1197.000000,2.677153e+05,2002,2499,...,0.0,0.102129,2499.0,2499.0,1221.0,51.0,2002.0,2002.0,0,both
99,f1_steam_2001_12_20_0_1,e. d. edwards,central illinois light company,coal,1972-01-01,1960-01-01,780.299988,3.791058e+06,2001,1582,...,3.0,131.090317,1582.0,1582.0,3887.0,56.0,2001.0,2001.0,0,both
100,f1_steam_2002_12_20_0_1,e. d. edwards,central illinois light company,coal,1972-01-01,1960-01-01,780.299988,3.536593e+06,2002,1582,...,4.0,525.004631,1582.0,1582.0,3887.0,56.0,2002.0,2002.0,0,both
101,f1_steam_2003_12_20_0_1,e. d. edwards,central illinois light company,coal,1972-01-01,1960-01-01,780.299988,3.014289e+06,2003,1582,...,0.0,0.102129,1582.0,1582.0,2066.0,56.0,2003.0,2003.0,0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82270,pumped_storage_generating_plant_statistics_lar...,bath county,virginia electric and power company,<NA>,1985-01-01,1985-01-01,1717.000000,2.772054e+06,2022,936,...,0.0,0.102129,936.0,936.0,349.0,349.0,2022.0,2022.0,0,both
82271,pumped_storage_generating_plant_statistics_lar...,wallace dam,georgia power company,<NA>,1980-01-01,1979-01-01,209.000000,1.248940e+05,2022,627,...,0.0,0.102129,627.0,627.0,123.0,123.0,2022.0,2022.0,0,both
82272,pumped_storage_generating_plant_statistics_lar...,rocky mountain,georgia power company,<NA>,1995-01-01,1995-01-01,278.000000,1.735853e+05,2022,917,...,0.0,0.102129,917.0,917.0,123.0,123.0,2022.0,2022.0,0,both
82275,pumped_storage_generating_plant_statistics_lar...,jocassee,duke energy carolinas limited liability company,<NA>,1975-01-01,1973-01-01,774.000000,1.206162e+06,2021,900,...,0.0,0.102129,900.0,900.0,90.0,90.0,2021.0,2021.0,0,both


In [86]:
incorrect_matches = pd.read_parquet("incorrect_matches.parquet")
sorted_preds_df = pd.read_parquet("sorted_preds_df.parquet")
eia_df = pd.read_parquet("eia_df.parquet")
ferc_df = pd.read_parquet("ferc_df.parquet")

i = 0
# ferc_id = incorrect_matches.record_id_ferc1.iloc[i]
ferc_id = "f1_steam_2003_12_20_0_1"
# true_eia_id = incorrect_matches.record_id_eia_true.iloc[i]
true_eia_id = "856_2003_plant_total_49756"
pred_eia_id = incorrect_matches.record_id_eia_pred.iloc[i]

rec_true = sorted_preds_df[
    (sorted_preds_df.record_id_r == ferc_id)
    & (sorted_preds_df.record_id_l == true_eia_id)
]
rec_pred = sorted_preds_df[
    (sorted_preds_df.record_id_r == ferc_id)
    & (sorted_preds_df.record_id_l == pred_eia_id)
]

rec_true = rec_true.to_dict(orient="records")
linker = DuckDBLinker(
    [eia_df, ferc_df],
    input_table_aliases=["eia_df", "ferc_df"],
)
linker.load_model("model_settings_splink_ferc_eia_demo.json")
chart = linker.waterfall_chart(rec_true, filter_nulls=False)

In [6]:
type(chart)

altair.vegalite.v5.api.LayerChart

In [84]:
chart

alt.LayerChart(...)

In [9]:
import altair as alt

In [12]:
alt.LayerChart(chart.data)

alt.LayerChart(...)